### Road Following - Live demo

In [2]:
#############################################
# Initialize the PyTorch model
#############################################

import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

print("Initialized the PyTorch model")

#############################################
# load the trained weights from the ``best_steering_model_xy.pth``
#############################################

model.load_state_dict(torch.load('best_steering_model_xy.pth'))

print("Loaded trained weights")

#############################################
# Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.
#############################################

device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

print("model weights are now in the GPU")

#############################################
# Creating the Pre-Processing Function
#############################################

# We have now loaded our model, but there's a slight issue. The format that we trained our model doesn't exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps:

# 1. Convert from HWC layout to CHW layout
# 2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
# 3. Transfer the data from CPU memory to GPU memory
# 4. Add a batch dimension

import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

print("Created the Pre-Processing Function")

#############################################
# let's start and display our camera.
#############################################

from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

print("Camera displayed")

#############################################
# create our robot instance which we'll need to drive the motors.
#############################################

from jetbot import Robot

robot = Robot()

print("Created robot instance")

#############################################
# Now, we will define sliders to control JetBot
#############################################

speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

print("Created sliders to control JetBot")

#############################################
# display some sliders that will let us see what JetBot is thinking.  The x and y sliders will display the predicted x, y values.
#############################################

x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

print("Created sliders to see what jetbot is thinking")

#############################################
# create a function that will get called whenever the camera's value changes (CONTROL PD)
#############################################

# This function will do the following steps

# 1. Pre-process the camera image
# 2. Execute the neural network
# 3. Compute the approximate steering value
# 4. Control the motors using proportional / derivative control (PD)

angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
execute({'new': camera.value})

print("Control PD ready")

#############################################
# attach the neural network execution to the camera for processing
#############################################

camera.observe(execute, names='value')

print("Jetbot running...")

Initialized the PyTorch model
Loaded trained weights
model weights are now in the GPU
Created the Pre-Processing Function


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Camera displayed
Created robot instance


FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

...

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

Created sliders to control JetBot


FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

Created sliders to see what jetbot is thinking
Control PD ready
Jetbot running...


### If you want to stop this behavior, you can unattach this callback by executing the code below.

In [1]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

camera.stop()

print("Jetbot stopped...")

NameError: name 'camera' is not defined